In [ ]:
import numpy as np
import pandas as pd

crime_raw_data = pd.read_csv("../data/2016년.csv", encoding="euc-kr")
crime_raw_data

In [ ]:
crime_raw_data["죄종"].unique()

In [ ]:
crime_raw_data=crime_raw_data[crime_raw_data["죄종"].notnull()]
crime_raw_data.info()

In [ ]:
crime_station = crime_raw_data.pivot_table(index="구분", columns=["죄종", "발생검거"], aggfunc=np.sum)
crime_station.head()

In [ ]:
crime_station.columns = crime_station.columns.droplevel([0])

In [ ]:
tmp = crime_station.columns.get_level_values(0) + crime_station.columns.get_level_values(1)
crime_station.columns=tmp
tmp

In [ ]:
crime_station.rename(columns={"강간검거":"성범죄검거","강간발생":"성범죄발생"}, inplace=True)

In [ ]:
crime_station.head()

In [ ]:
real_gu = {'종로구': ['종로', '혜화'], '중구': ['남대문', '중부'],
           '용산구': ['용산'], '성동구': ['성동'],
           '광진구': ['광진'], '동대문구': ['동대문'],
           '중랑구': ['중랑'], '성북구': ['성북', '종암'],
           '강북구': ['강북'], '도봉구': ['도봉'],
           '노원구': ['노원'], '은평구': ['서부', '은평'],
           '서대문구': ['서대문'], '마포구': ['마포'],
           '양천구': ['양천'], '강서구': ['강서'],
           '구로구': ['구로'], '금천구': ['금천'],
           '영등포구': ['영등포'], '동작구': ['동작'],
           '관악구': ['관악'], '서초구': ['방배', '서초'],
           '강남구': ['강남', '수서'], '송파구': ['송파'],
           '강동구': ['강동']}

In [ ]:
tmp_gu = []

for station in crime_station.index:
    for gu_name, station_name in real_gu.items():
        if station in station_name:
            tmp_gu.append(gu_name)

In [ ]:
tmp_gu

In [ ]:
crime_station['구'] = tmp_gu
crime_station.head()

In [ ]:
crime_gu = crime_station.pivot_table(index="구", aggfunc=np.sum)

In [ ]:
crime_gu["강도발생"].sort_values(ascending=False).head()

In [ ]:
arrest_col = ["강도검거", "살인검거", "성범죄검거", "절도검거", "폭력검거"]
occur_col = ["강도발생", "살인발생", "성범죄발생", "절도발생", "폭력발생"]
crime_gu[arrest_col].div(crime_gu[occur_col].values) * 100
arrest_rate = ["강도검거율", "살인검거율", "성범죄검거율", "절도검거율", "폭력검거율"]
crime_gu[arrest_rate] = crime_gu[arrest_col].div(crime_gu[occur_col].values)*100
crime_gu.head()

In [ ]:
crime_gu[crime_gu[arrest_rate] > 100] = 100
crime_gu.head(3)

In [ ]:
crime_gu.drop(columns=arrest_col, axis=1, inplace=True)

In [ ]:
crime_gu.rename(columns={
    "강도발생":"강도",
    "살인발생":"살인",
    "성범죄발생":"성범죄",
    "절도발생":"절도",
    "폭력발생":"폭력",
    }, inplace=True)
crime_gu.head()

In [ ]:
occur_col = ["강도", "살인", "성범죄", "절도", "폭력"]

In [ ]:
crime_gu_norm = crime_gu[occur_col] / crime_gu[occur_col].max()
crime_gu_norm.head()

In [ ]:
crime_gu_norm[arrest_rate] = crime_gu[arrest_rate]
crime_gu_norm.head()

In [ ]:
crime_gu_norm["범죄"] = np.mean(crime_gu_norm[occur_col], axis=1)

In [ ]:
crime_gu_norm["검거"] = np.mean(crime_gu_norm[arrest_rate], axis=1)

In [ ]:
crime_gu_norm["범죄"].sort_values(ascending=False).head()

In [ ]:
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns

In [ ]:
crime_gu_norm_sort = crime_gu_norm.sort_values(by="검거", ascending=False)
crime_gu_norm_sort.head()

In [ ]:
target_col = ["강도검거율", "살인검거율", "성범죄검거율", "절도검거율", "폭력검거율", "검거"]

plt.figure(figsize=(10,10))
sns.heatmap(crime_gu_norm_sort[target_col],annot=True, fmt='f', lw=.5, cmap='RdPu')
plt.title("범죄 검거 비율")
plt.ylabel('')
plt.show()

In [ ]:
target_col = ["강도", "살인", "성범죄", "절도", "폭력", "범죄"]
crime_gu_norm_sort = crime_gu_norm.sort_values(by="범죄", ascending=False)

plt.figure(figsize=(10,10))
sns.heatmap(crime_gu_norm_sort[target_col],annot=True, fmt='f', lw=.5, cmap='RdPu')
plt.title("범죄 검거 비율")
plt.ylabel('')
plt.show()

In [ ]:
crime_gu_norm.to_csv("../data/seoul_crime_result.csv",sep=",",encoding="utf-8")